In [72]:
import numpy as np

class MultivariateNormal:
    """
    Class of multivariate normal distribution.

    Parameters
    ----------
    μ: ndarray(float, dim=1)
        the mean of z, N by 1
    Σ: ndarray(float, dim=2)
        the covarianece matrix of z, N by 1

    Arguments
    ---------
    μ, Σ:
        see parameters
    μs: list(ndarray(float, dim=1))
        list of mean vectors μ1 and μ2 in order
    Σs: list(list(ndarray(float, dim=2)))
        2 dimensional list of covariance matrices
        Σ11, Σ12, Σ21, Σ22 in order
    βs: list(ndarray(float, dim=1))
        list of regression coefficients β1 and β2 in order
    """

    def __init__(self, μ, Σ):
        "initialization"
        self.μ = np.array(μ)
        self.Σ = np.atleast_2d(Σ)

    def partition(self, k):
        """
        Given k, partition the random vector z into a size k vector z1
        and a size N-k vector z2. Partition the mean vector μ into
        μ1 and μ2, and the covariance matrix Σ into Σ11, Σ12, Σ21, Σ22
        correspondingly. Compute the regression coefficients β1 and β2
        using the partitioned arrays.
        """
        μ = self.μ
        Σ = self.Σ

        self.μs = [μ[:k], μ[k:]]
        self.Σs = [[Σ[:k, :k], Σ[:k, k:]],
                   [Σ[k:, :k], Σ[k:, k:]]]

        self.βs = [self.Σs[0][1] @ np.linalg.inv(self.Σs[1][1]),
                   self.Σs[1][0] @ np.linalg.inv(self.Σs[0][0])]

    def cond_dist(self, ind, z):
        """
        Compute the conditional distribution of z1 given z2, or reversely.
        Argument ind determines whether we compute the conditional
        distribution of z1 (ind=0) or z2 (ind=1).

        Returns
        ---------
        μ_hat: ndarray(float, ndim=1)
            The conditional mean of z1 or z2.
        Σ_hat: ndarray(float, ndim=2)
            The conditional covariance matrix of z1 or z2.
        """
        β = self.βs[ind]
        μs = self.μs
        Σs = self.Σs

        μ_hat = μs[ind] + β @ (z - μs[1-ind])
        Σ_hat = Σs[ind][ind] - β @ Σs[1-ind][1-ind] @ β.T

        return μ_hat, Σ_hat

    
μ = np.array([0., 0.])
Σ = np.array([[1., .2], [.2 ,1.]])

# construction of the multivariate normal instance
multi_normal = MultivariateNormal(μ, Σ)

In [73]:
k = 1 # choose partition

# partition and compute regression coefficients
multi_normal.partition(k)
multi_normal.βs[0]

array([[0.2]])

In [74]:
# compute the cond. dist. of z1
ind = 0
z2 = np.array([5.]) # given z2

μ1_hat, Σ1_hat = multi_normal.cond_dist(ind, z2)
print('μ1_hat, Σ1_hat = ', μ1_hat, Σ1_hat)

μ1_hat, Σ1_hat =  [1.] [[0.96]]


In [79]:
import statsmodels.api as sm

n = 1_000_000 # sample size

# simulate multivariate normal random vectors
data = np.random.multivariate_normal(μ, Σ, size=n)
z1_data = data[:, 0]
z2_data = data[:, 1]

# OLS regression
μ1, μ2 = multi_normal.μs
results = sm.OLS(z1_data - μ1, z2_data - μ2).fit()

multi_normal.βs[0], results.params


(array([[0.2]]), array([0.19933719]))

In [80]:
Σ1_hat, results.resid @ results.resid.T / (n - 1)


(array([[0.96]]), 0.9587936611538904)

In [81]:
μ1_hat, results.predict(z2 - μ2) + μ1


(array([1.]), array([0.99668593]))

In [70]:
μ = np.random.random(3)
C = np.random.random((3, 3))
Σ = C @ C.T # positive semi-definite

multi_normal = MultivariateNormal(μ, Σ)


In [64]:
mean = np.array([1, 2, 3, 4])
cov = np.array(
    [[ 1.0,  0.5,  0.3, -0.1], 
     [ 0.5,  1.0,  0.1, -0.2], 
     [ 0.3,  0.1,  1.0, -0.3], 
     [-0.1, -0.2, -0.3,  0.1]])  # diagonal covariance

mv = MultivariateNormal(mean, covv)
mv.partition(2)
# mv.cond_dist(0, 2)
mv.μs, mv.Σs

([array([1, 2]), array([3, 4])],
 [[array([[1.        , 0.33333333],
          [0.33333333, 1.        ]]), array([], shape=(2, 0), dtype=float64)],
  [array([], shape=(0, 2), dtype=float64),
   array([], shape=(0, 0), dtype=float64)]])

In [26]:
mean = [1, 2]
covv = [[1, 1/3],
        [1/3, 1]]

mv = MultivariateNormal(mean, covv)
mv.partition(1)
mv.cond_dist(0, 2)


(array([1.]), array([[0.88888889]]))

In [98]:
mean = np.array([1, 2, 3, 4])
covv = np.array(
    [[ 1.0,  0.5,  0.3, -0.1], 
     [ 0.5,  1.0,  0.1, -0.2], 
     [ 0.3,  0.1,  1.0, -0.3], 
     [-0.1, -0.2, -0.3,  0.1]])  # diagonal covariance


mv = MultivariateNormal(mean, covv)
mv.partition(1)
mv.cond_dist(0, [1, 1, 1])

(array([13.78947368]), array([[0.94736842]]))

In [94]:
mean = [1, 2, 3]
covv = [[1.0,  0.5,  0.3], 
       [0.5,  1.0,  0.1],
       [0.3,  0.1,  1.0]]


# conditional_MvNormal(mean, covv, [2, 3], [2, 3])
# ([1.0], [0.6868686868686869])

mv = MultivariateNormal(mean, covv)
mv.partition(1)
mv.cond_dist(0, [2, 3])

(array([1.]), array([[0.68686869]]))

In [82]:
μ = np.random.random(3)
C = np.random.random((3, 3))
Σ = C @ C.T # positive semi-definite

multi_normal = MultivariateNormal(μ, Σ)

In [83]:
μ, Σ

(array([0.25614479, 0.65047708, 0.77535447]),
 array([[1.49353187, 1.37326306, 0.560267  ],
        [1.37326306, 1.36990644, 0.77764709],
        [0.560267  , 0.77764709, 0.87810087]]))

In [84]:
k = 1
multi_normal.partition(k)


In [85]:
ind = 0
z2 = np.array([2., 5.])

μ1_hat, Σ1_hat = multi_normal.cond_dist(ind, z2)


In [86]:
n = 1_000_000
data = np.random.multivariate_normal(μ, Σ, size=n)
z1_data = data[:, :k]
z2_data = data[:, k:]



In [87]:
μ1, μ2 = multi_normal.μs
results = sm.OLS(z1_data - μ1, z2_data - μ2).fit()


In [88]:
multi_normal.βs[0], results.params


(array([[ 1.28752576, -0.50219021]]), array([ 1.2875541, -0.5021211]))